# Обработка данных - Промышленная недвижимость

## 1. Загружаем файлы и объединяем в один датафрейм

In [222]:
# Расширяем режим просмотра Jupyter
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 90% !important; }</style>"))

In [223]:
# Загружаем библиотеки

# Стандартные библиотеки
import os
import random
import re

# Сторонние библиотеки
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import pymorphy2
from rutermextract import TermExtractor
from tqdm import tqdm

# Инициализация анализатора, экстрактора терминов и отображение прогресс бара в pandas
morph = pymorphy2.MorphAnalyzer()
term_extractor = TermExtractor()
tqdm.pandas()

In [224]:
# Константы

# Путь к файлам на рабочем компьютере
DIR = 'L:/Беспалов Р.А/Обработка рынка Брянск/'

# Путь к исходным данным
PATH_INITIAL_DATA = os.path.join(DIR, 'Исходники рынок Брянск - промышленная недвижимость/')

# Путь к папке для хранения очищенных данных
PATH_CLEAN_DATA = os.path.join(DIR, 'Очищенные данные/')

# Путь к вспомогательным файлам
PATH_SUPPORT_DATA = os.path.join(DIR, 'Вспомогательные файлы/')

# Путь хранения найденных адрес-координат
PATH_ADDRESS_COORD_DATA = os.path.join(DIR, 'Адрес-координаты/')

# Путь к словарям
PATH_DICT_DATA = os.path.join(DIR, 'Словари/')

# Дополнительные данные для тестов
PATH_TEST_DATA = os.path.join(DIR, 'Данные для тестов/')

In [225]:
# Установим рабочий каталог
os.chdir(DIR)

In [226]:
# Получаем список файлов
files = os.listdir(PATH_INITIAL_DATA)

In [227]:
def sum_data():
    """Создает датафрейм из исходных данных."""
    # Создаем список для хранения датафреймов
    data = []

    # Итерируемся по каждому файлу
    for file in tqdm(files):
        if file.split('.')[-1] == 'xlsx':
            # Создаем путь к каждму файлу
            path_file = os.path.join(PATH_INITIAL_DATA, file)
            # добавляем в список каждый датафрейм
            table = pd.read_excel(path_file)
            table['Имя файла'] = file
            data.append(table)
    
    # Объединяем в один Датафрейм и обнуляем индексы
    df = pd.concat(data, axis=0, ignore_index=True)
    print('Количество строк df =', len(df))
    return df

df = sum_data()

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:54<00:00,  3.00s/it]

Количество строк df = 27923


In [228]:
def sum_test_data():
    """Создает датафрейм из исходных и тестовых данных."""
    # Создаем список для хранения датафреймов
    data = []
    # путь к тестовым данным
    path_test_data = os.path.join(PATH_TEST_DATA, 'проверить_квартиры_v13.xlsx')
    # Итерируемся по каждому файлу
    for file in tqdm(files):
        if file.split('.')[-1] == 'xlsx':
            # Создаем путь к каждму файлу
            path_file = os.path.join(PATH_INITIAL_DATA, file)
            # добавляем в список каждый датафрейм
            data.append(pd.read_excel(path_file))
    # добавляем тестовые данные
    data.append(pd.read_excel(path_test_data))
    # Объединяем в один Датафрейм и обнуляем индексы
    df = pd.concat(data, axis=0, ignore_index=True)
    print('Количество строк df_777 =', len(df))
    return df

## 2. Очищаем и фильтруем данные

In [229]:
def clean_data(df):
    """Унифицируем поля"""
    # в столбце "Текст объявления" встречаются ячейки, начинающиеся со знака "=",
    # поэтому при выгрузке в эксель файл, он думает, что это формула, поэтому исправляем, убирая знак "="
    df['Текст объявления'] = df['Текст объявления'].str.replace(r'^=', '', regex=True)
    
    # Унифицируем дату
    df['Дата объявления'] = pd.to_datetime(df['Дата объявления'], dayfirst=True)
    df['Дата предложения'] = pd.to_datetime(df['Дата предложения'], dayfirst=True)
    
    # Преобразовываем тип данных object в float
    df['Площадь объекта(м²)'] = df['Площадь объекта(м²)'].replace(',', '.', regex=True)
    df['Площадь объекта(м²)'] = df['Площадь объекта(м²)'].astype('float64')
    
    df['Цена(₽/м²)'] = df['Цена(₽/м²)'].replace(',', '.', regex=True)
    df.loc[df['Цена(₽/м²)'].str.lower() == 'смотреть цена указана за кв. м или за весь объект', 'Цена(₽/м²)'] = None
    df['Цена(₽/м²)'] = df['Цена(₽/м²)'].astype('float64')
    
    df['Цена(₽)'] = df['Цена(₽)'].astype('float64')
    return df

df = clean_data(df)

In [230]:
# Статистика
control_count_result = pd.pivot_table(df, 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': 'Исходное количество записей'})

In [231]:
def emissions_1(df):
    """Исключает и выделяет в отдельные файлы по параметрам"""
    # гараж, машиноместо или паркинг выделяем в отдельный файл
    df_garage = df[df['Категория'] == 'гараж, машиноместо или паркинг']
    # df_garage.to_excel(PATH_CLEAN_DATA + 'гараж, машиноместо или паркинг.xlsx', index=False)
    
    # исключаем гараж, машиноместо или паркинг из общего датафрейма
    df = df[df['Категория'] != 'гараж, машиноместо или паркинг']
    return df

def emissions_2(df):
    # Исключаем строки для которых значение столбца 'Тип предложения' == 'аренда'
    df = df.loc[df['Тип предложения'] != 'аренда']
    
    # Сброс индексов, drop=True - удаляет старые индексы
    df = df.reset_index(drop=True)
    print('Количество строк df =', len(df))
    return df

In [232]:
# Статистика
control_count_temp = pd.pivot_table(df[df['Категория'] == 'гараж, машиноместо или паркинг'], 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': 'Выбросы гараж, машиноместо или паркинг'})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)

In [233]:
df = emissions_1(df)

In [234]:
# Статистика
control_count_temp = pd.pivot_table(df.loc[df['Тип предложения'] == 'аренда'], 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': 'Предложения аренда'})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)

In [235]:
df = emissions_2(df)

Количество строк df = 1764


In [236]:
def check_price(df):
    """Проверка 'Цена(₽)' на 0 и на пустое значение."""

    # Булева операция на поиск пустых ячеек в столбце 'Цена(₽)'
    mask_none_price = df['Цена(₽)'].isnull() # это булевая серия

    # Булева операция на поиск значения 0 в ячейках в столбце 'Цена(₽)'
    mask_0_price = df['Цена(₽)'] == 0 # это булевая серия

    messages = []
    if mask_none_price.any():
        messages.append('Найдены пустые значения в столбце "Цена(₽)"')
    if mask_0_price.any():
        messages.append('Найдены нулевые значения в столбце "Цена(₽)"')
    if not messages:
        messages.append('Пустых или нулевых значений в столбце "Цена(₽)" нет!')
    return '\n'.join(messages)

print(check_price(df))

Пустых или нулевых значений в столбце "Цена(₽)" нет!


In [237]:
def check_area(df):
    """Проверка 'Площадь объекта(м²)' на 0 и на пустое значение."""
    # Булева операция на поиск пустых ячеек в столбце 'Цена(₽)'
    mask_none_area = df['Площадь объекта(м²)'].isnull() # это булевая серия
    
    # Булева операция на поиск значения 0 в ячейках в столбце 'Цена(₽)'
    mask_0_area = df['Площадь объекта(м²)'] == 0 # это булевая серия

    messages = []
    if mask_none_area.any():
        messages.append('Найдены пустые значения в столбце "Площадь объекта(м²)"')
    if mask_0_area.any():
        messages.append('Найдены нулевые значения в столбце "Площадь объекта(м²)"')
    if not messages:
        messages.append('Пустых или нулевых значений в столбце "Площадь объекта(м²)" нет!')
    print('\n'.join(messages))
    return mask_none_area

mask_none_area = check_area(df)

Найдены пустые значения в столбце "Площадь объекта(м²)"


In [238]:
def find_area_by_price_square(df):
    """Проверяет поля для определения площади."""
    # Пытаемся определить площадь, если известна цена за 1 кв.м и общая стоимость
    # площадь = общая стоимость/цена за 1 кв.м
    df_area_none = df[mask_none_area]
    df_area_none = df_area_none[(df_area_none['Цена(₽/м²)'] != 0) & (df_area_none['Цена(₽/м²)'].notnull())]
    if len(df_area_none):
        return 'Есть данные'
    else:
        return 'Нет параметра "Цена(₽/м²)" для того чтобы определить площадь'

print(find_area_by_price_square(df))

Нет параметра "Цена(₽/м²)" для того чтобы определить площадь


In [239]:
def save_none_area(df):
    """Создаем отдельный датафрейм, значения строк у которых 'Площадь объекта(м²)' является пустым."""
    df_area_none = df[mask_none_area]
    df_area_none = df_area_none.reset_index(drop=True)
    # Сохраняем датафрейм в ексель файл "Промышленная_недвижимость_без_площади.xlsx"
    # df_area_none.to_excel(PATH_CLEAN_DATA + 'промышленная_недвижимость_без_площади.xlsx', index=False)
    print('Количество строк df_area_none =', len(df_area_none))

save_none_area(df)

Количество строк df_area_none = 126


In [240]:
# Статистика
control_count_temp = pd.pivot_table(df[df['Площадь объекта(м²)'].isnull()], 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': 'Записи без площади'})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)

In [241]:
def update(df):
    # Исключаем из результирующего датафрейма пустые значения для столбца 'Площадь объекта(м²)'
    df = df[df['Площадь объекта(м²)'].notnull()]
    df = df.reset_index(drop=True)
    print('Количество строк df =', len(df))
    return df

df = update(df)

Количество строк df = 1638


In [242]:
def columns_to_str(df):
    """Заменяет None на '-' и меняет тип данных на 'str'."""
    columns = ['Регион', 'Район субъекта РФ', 'Город', 'Населенный пункт', 'Улица', 'Номер дома', 'Координаты объекта', 'Блок', 'Городской район']
    for column in tqdm(columns):
        df[column] = df[column].fillna('-')
        df[column] = df[column].astype('str')
    return df

df = columns_to_str(df)

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 542.10it/s]


## 3. Распространение координат по имеющимся данным

In [243]:
def str_coord_to_tuple(string):
    """Конвертирует строку координат в кортеж."""
    try:
        lat = string.split(',')[0].split(':')[-1]
        lon = string.split(',')[1].split(':')[-1][:-1]
        return tuple((float(lat), float(lon)))
    except IndexError:
        print(f"Ошибка обработки данных: {string}")
        return None

In [244]:
def spread_coord(df):
    """Распространение координат Брянска по адресам."""
    # применение функции str_coord_to_tuple
    df['Координаты объекта'] = df['Координаты объекта'].apply(lambda x: str_coord_to_tuple(x) if x != '-' else x)
    # берем только те адреса, где есть координаты
    df_address = df[df['Координаты объекта'] != '-']
    # создаем словарь
    dict_coordinates = dict(df_address[['Адрес из источника', 'Координаты объекта']].to_numpy())
    # распространяем координаты Брянска на соответствующие адреса
    df['Координаты объекта'] = df['Адрес из источника'].apply(lambda x: dict_coordinates.get(x, '-'))
    return df

df = spread_coord(df)

In [245]:
def useless_data(df):
    """Исключает объекты без адреса и координат."""
    indices_to_remove = []
    for i in tqdm(range(len(df))):
        if (df['Улица'].iloc[i] == '-' or df['Номер дома'].iloc[i] == '-') and df['Координаты объекта'].iloc[i] == '-':
            indices_to_remove.append(i)

    df_none_address = df.loc[indices_to_remove]
    # df_none_address.to_excel(PATH_CLEAN_DATA + 'промышленная_недвижимость_без_адреса_и_координат.xlsx', index=False)
    return indices_to_remove

indices_to_remove = useless_data(df)

100%|███████████████████████████████████████████████████████████████████████████| 1638/1638 [00:00<00:00, 19818.58it/s]


In [246]:
# Статистика
control_count_temp = pd.pivot_table(df.loc[indices_to_remove], 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': 'Записи без улицы или дома (невозможно определить координаты)'})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)

In [247]:
def drop_adress(df):
    df = df.drop(indices_to_remove)
    df = df.reset_index(drop=True)
    print(len(df))
    return df

df = drop_adress(df)

1553


## 4. Выгрузка координат с помощью геокодирования по адресу

### 4.1. Проверка поселков на вхождение в НП

In [248]:
# Константы
PRIMORSKY_KRAI_CITIES = 'города.xlsx'
PRIMORSKY_KRAI_SETTLEMENTS = 'Населенные_пункты_полигоны.xlsx'

In [249]:
def clean_settlements(file_name):
    """Извлекает населенные пункты и возвращает их в очищенном виде."""
    file_path = os.path.join(PATH_SUPPORT_DATA, file_name)
    df = pd.read_excel(file_path)

    df['NAME'] = df['NAME'].str.lower()
    df['NAME'] = df['NAME'].str.replace('ё', 'е')

    return set(df['NAME'])


def get_villages(data_1, data_2):
    """Удаляет из населенных пунктов города и возвращает поселки."""
    settlements = clean_settlements(data_1)
    cities = clean_settlements(data_2)

    villages = settlements - cities # все поселки Приморского края

    return villages

# Получение поселков
villages = get_villages(PRIMORSKY_KRAI_SETTLEMENTS, PRIMORSKY_KRAI_CITIES)

In [250]:
def clean_columns(df):
    """Приводит столбец к унифицированному виду и определяет деревни/поселки/села."""

    df['Населенный пункт'] = df['Населенный пункт'].str.lower()
    df['Населенный пункт'] = df['Населенный пункт'].str.replace('поселок ', '')
    df['Населенный пункт'] = df['Населенный пункт'].str.replace('ё', 'е')

    mask = df['Населенный пункт'].isin(villages)
    # Создаем столбец 'Населенный пункт temp' в df, где записываем значение 'Населенный пункт' или '-' в зависимости от маски
    df['Населенный пункт temp'] = df['Населенный пункт'].where(mask, other='-')

    return df

df = clean_columns(df)

### 4.2. Собираем адрес из детальной раскладки

In [251]:
def region(df):
    """Определяет городской округ по городу для поля «Район субъекта РФ»."""
    region_dict = {
        'Арсеньев': 'Арсеньевский', 'Артем': 'Артёмовский', 'Большой Камень': 'Большой Камень',
        'Владивосток': 'Владивостокский', 'Дальнегорск': 'Дальнегорский',
        'Дальнереченск': 'Дальнереченский', 'Лесозаводск': 'Лесозаводский',
        'Находка': 'Находкинский', 'Партизанск': 'Партизанский',
        'Спасск-Дальний': 'Спасск-Дальний', 'Уссурийск': 'Уссурийский',
        'Фокино': 'ЗАТО Фокино',
    }

    for city, region in region_dict.items():
        df.loc[(df['Город'] == city) & (df['Район субъекта РФ'] == '-'), 'Район субъекта РФ'] = region

    return df

df = region(df)

In [252]:
def del_villages_1(df):
    """Функция для тестов."""
    df['Городской район'] = df['Городской район'].str.lower()
    df['Городской район'] = df['Городской район'].str.replace('ё', 'е')
    df = df[df['Населенный пункт temp'] == '-']
    return df

In [253]:
def find_villages(data):
    """Функция для тестов."""
    """Определяет является ли строка в поле «Городской район» населенным пунктом."""
    # создаем шаблон для разделения строки
    pattern = r'[ ,.\-:;!]+'

    # находим чистую строку с поселком
    if data in villages:
        return data
    else:
        # определяет только полное совпадение (работает когда поселок это одно слово)
        # + проверка на ошибки 1 рода, создадим слова исключения
        # list_exceptions = ['микрорайон', 'мкр', 'мкр', 'м', 'им', 'ул', 'автовокзал', 'база']
        words = re.split(pattern, data)
        for word in words:
            # if word in list_exceptions:
            #     return '-'
            word = {word}
            if word.intersection(villages):
                return ' '.join(word.intersection(villages))

        # проверяет что хотя бы одно слово пересекается с названием населенного пункта, тогда проверяется ближайшее слово (+-1) вместе с найденным на полное соответствие населенному пункту
        for word in words:
            for village in villages:
                lst = re.split(pattern, village)
                if word in lst:
                    index = words.index(word)
                    length = len(words) - 1
                    if 0 <= index - 1 <= length:
                        if (words[index - 1] + ' ' + words[index] == village) or (words[index - 1] + '-' + words[index] == village):
                            return village
                    if 0 <= index + 1 <= length:
                        if (words[index] + ' ' + words[index + 1] == village) or (words[index] + '-' + words[index + 1] == village):
                            return village
    return '-'

In [254]:
def del_villages_2(df):
    """Функция для тестов."""
    df['Городской район temp'] = df['Городской район'].progress_apply(lambda x: find_villages(x) if x != '-' else '-')
    df = df[df['Городской район temp'] != '-']
    df = df[df['Координаты объекта'] != '-']
    return df

In [255]:
def find_coord(df):
    """Создает отдельный датафрейм из данных в которых нет координат."""
    df = df[df['Координаты объекта'] == '-']
    
    # Удаляем дубликаты по столбцу 'Адрес из источника'
    df = df.drop_duplicates('Адрес из источника')
    df = df.reset_index(drop=True)
    print(len(df))
    return df

df_address_coord = find_coord(df)

164


In [256]:
def create_detail_address(data):
    """Функция создания адреса из детальной раскладки."""
    addresses_list = []
    for i in tqdm(range(len(data))):
        region = data['Регион'].iloc[i]
        district = data['Район субъекта РФ'].iloc[i]
        city = data['Город'].iloc[i]
        settlement = data['Населенный пункт'].iloc[i]
        street = data['Улица'].iloc[i]
        house_number = data['Номер дома'].iloc[i]
        street_address = ""

        block = data['Блок'].iloc[i].split('.')[0]
        
        # сборка поселков
        if data['Населенный пункт temp'].iloc[i] != '-' and data['Населенный пункт'].iloc[i] != '-':
            street_address = f"{region} край, {district}, {settlement}, {street}, {house_number}"
        # # сборка поселков из Городского района
        # elif data['Городской район temp'].iloc[i] != '-':
        #     street_address = f"{region} край, {district}, {data['Городской район temp'].iloc[i]}, {street}, {house_number}"
        #сборка города
        else:
            street_address = f"{region} край, {city}, {street}, {house_number}"

        # когда есть / и блок
        if (not pd.isna(data['Тип блока'].iloc[i])) and ('/' in data['Адрес из источника'].iloc[i]):
            digit_1 = data['Адрес из источника'].iloc[i].split('/')[0].strip().split()[-1]
            digit_2 = data['Адрес из источника'].iloc[i].split('/')[-1].strip().split()[0]
            number = f"{digit_1}/{digit_2}"
            street_address = street_address.rsplit(', ', 1)[0]
            if digit_1 == block or digit_2 == block:
                street_address += f" {number}"
            else:
                street_address += f" {number} {data['Тип блока'].iloc[i][:1]}{block}"

        # строение
        elif data['Тип блока'].iloc[i] == 'строение':
            street_address += f" с{block}"
        
        # корпус
        elif data['Тип блока'].iloc[i] == 'корпус':
            street_address += f" к{block}"
        
        # там где забыли указать /номер дома
        elif ('/' in data['Адрес из источника'].iloc[i]) and (pd.isna(data['Тип блока'].iloc[i])) and ('/' not in data['Номер дома']):
            if len(data['Адрес из источника'].iloc[i].split('/')[-1]) > 0:
                number = data['Адрес из источника'].iloc[i].split('/')[-1].strip().split()[0]
                street_address += f"/{number}"

        # случай когда не укзан тип блока но при этом есть "строние" в адресе
        elif ('стр.' in data['Адрес из источника'].iloc[i]) and (pd.isna(data['Тип блока'].iloc[i])):
            if len(data['Адрес из источника'].iloc[i].split('стр.')[-1]) > 0:
                stroyenie = data['Адрес из источника'].iloc[i].split('стр.')[-1].strip().split()[0]
                number = data['Номер дома'].iloc[i].split('/')[0]
                street_address = street_address.rsplit(', ', 1)[0]
                street_address += f", {number} с{stroyenie}"

        # случай когда не укзан тип блока но при этом есть "корус" в адресе
        elif ('кор.' in data['Адрес из источника'].iloc[i]) and (pd.isna(data['Тип блока'].iloc[i])):
            if len(data['Адрес из источника'].iloc[i].split('кор.')[-1]) > 0:
                korpus = data['Адрес из источника'].iloc[i].split('кор.')[-1].strip().split()[0]
                number = data['Номер дома'].iloc[i].split('/')[0]
                street_address = street_address.rsplit(', ', 1)[0]
                street_address += f", {number} к{korpus}"

        addresses_list.append(street_address)

    return addresses_list

In [257]:
def save_test(df):
    """Функция для тестов."""
    df.insert(7, 'Адрес детальной раскладки', create_detail_address(df))
    df.to_excel(PATH_CLEAN_DATA + 'Статистика для поиска неверных координат.xlsx', index=False)
    return df

### 4.3. Геокодируем адрес

In [258]:
initial_address_list = list(set(create_detail_address(df_address_coord))) # адреса полученные из детальной раскладки
nominatim_coord_list = list() # координаты по адресу
nominatim_address_list = list() # адрес который получен из координат
solution_list = list() # решение

100%|██████████████████████████████████████████████████████████████████████████████| 164/164 [00:00<00:00, 3486.64it/s]


In [259]:
# Проверка, что для адреса уже найдены координаты
def find_existing_address(df):
    existing_address_path = os.path.join(PATH_ADDRESS_COORD_DATA, 'адрес_координаты_1.xlsx')
    df_existing_address = pd.read_excel(existing_address_path)
    dict_existing_address = dict(df_existing_address[['Адрес из детальной раскладки', 'Координаты nominatim']].to_numpy())

    # итерируемся по адресам где нет координат
    for address in initial_address_list:
        if address in dict_existing_address:
            # удаляем адрес, если для него есть координаты (то есть его не будет в поиске, добавим его напрямую из имеющегося словаря)
            initial_address_list.remove(address)
    print(len(initial_address_list))
    return initial_address_list, dict_existing_address
# initial_address_list, dict_existing_address = find_existing_address(df_address_coord)

In [260]:
def get_coordinates(address):
    """Функция поиска координат по адресу."""
    global nominatim_coord_list, nominatim_address_list, solution_list
    geolocator = Nominatim(user_agent="request_vl_" + str(random.randint(0, 10000)))
    locations = geolocator.geocode(address, exactly_one=False, timeout=20)
    temp = []
    if locations:
        for location in locations:
            temp.append(location[0])
            # выбираем из предложенных вариантов то, где первым будет стоять номер дома, а не название организации
            index = np.argsort(temp)[0]
        if locations[index].raw['name'] != '':
            start_index = locations[index].address.find(locations[index].raw['name']) + len(locations[index].raw['name']) + 2
            address_nom = locations[index].address[start_index:]
        else:
            address_nom = locations[index].address

        nominatim_coord_list.append(locations[index][-1])
        nominatim_address_list.append(address_nom)
        if address_nom[0].isdigit():
            solution_list.append('да')
        else:
            solution_list.append('нет')
    else:
        nominatim_coord_list.append('-')
        nominatim_address_list.append('-')
        solution_list.append('-')

In [261]:
def fill_coord(df):
    """Определяет и сохраняет координаты в датафрейм."""
    global initial_address_list
    for address in tqdm(initial_address_list):
        get_coordinates(address)
    df_address_coord = pd.DataFrame({
        'Адрес из детальной раскладки': initial_address_list,
        'Координаты nominatim': nominatim_coord_list,
        'Адрес nominatim': nominatim_address_list,
        'Адрес верный': solution_list,
    })
    return df_address_coord

df_address_coord = fill_coord(df_address_coord)
# df_address_coord['Координаты nominatim'] = df_address_coord['Координаты nominatim'].apply(lambda coord: (round(coord[0], 5), round(coord[1], 5)))

100%|████████████████████████████████████████████████████████████████████████████████| 158/158 [04:47<00:00,  1.82s/it]


In [262]:
def add_coord(df, df_address_coord):
    """Затягивает на весь датафрейм координаты."""
    # Создадим временный столбец 'Адрес из детальной раскладки'
    # С помощью словаря затянем по этому столбцу найденные координаты
    df_address_coord = df_address_coord[df_address_coord['Адрес верный'] == 'да']
    
    dict_address_coord = dict(df_address_coord[['Адрес из детальной раскладки', 'Координаты nominatim']].to_numpy())
    
    full_address_list = create_detail_address(df)
    df['Адрес из детальной раскладки temp'] = full_address_list
    
    # Распространяем имеющиеся координаты на соответствующие адреса
    df['Координаты объекта'] = df.apply(lambda row: dict_address_coord[row['Адрес из детальной раскладки temp']] if row['Адрес из детальной раскладки temp'] in dict_address_coord else row['Координаты объекта'], axis=1)
    return df

def fill_existing_coord(df, dict_existing_address):
    df['Координаты объекта'] = df.apply(lambda row: dict_existing_address[row['Адрес из детальной раскладки temp']] if row['Адрес из детальной раскладки temp'] in dict_existing_address else row['Координаты объекта'], axis=1)
    return df

def edit_df(df):
    # Исключаем из итогового df объекты без координат
    df = df[df['Координаты объекта'] != '-']

    # Удаляем временные столбцы
    df = df.drop('Населенный пункт temp', axis=1)
    df = df.drop('Адрес из детальной раскладки temp', axis=1)
    return df

df = add_coord(df, df_address_coord)
# df = fill_existing_coord(df, dict_existing_address)

100%|████████████████████████████████████████████████████████████████████████████| 1553/1553 [00:00<00:00, 3144.65it/s]


In [263]:
# Статистика
control_count_temp = pd.pivot_table(df[df['Координаты объекта'] == '-'], 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': 'Записи для которых не удалось определить координаты'})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)

In [264]:
df = edit_df(df)

In [265]:
# import datetime


def save_coord():
    # current_time = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
    # Сохранение адрес-координаты в ексель
    df_temp = df_address_coord[df_address_coord['Адрес верный'] == 'да']
    df_temp = df_temp[['Адрес из детальной раскладки', 'Координаты nominatim']]
    # df_temp.to_excel(PATH_ADDRESS_COORD_DATA + "адрес_координаты_1.xlsx", index=False)

    # Адреса по которым не удалось определить координаты, либо полученные координаты не соответствуют адресу
    df_other = df_address_coord[df_address_coord['Адрес верный'] != 'да' ]
    # df_other.to_excel(PATH_CLEAN_DATA + 'не_удалось_найти_координаты_по_адресу.xlsx', index=False)

save_coord()

In [266]:
"""Функции для работы с тестовыми данными"""
def add_coord(df, df_address_coord):
    df_address_coord = df_address_coord[df_address_coord['Адрес верный'] == 'да']
    df_address_coord = df_address_coord[df_address_coord['Координаты nominatim'] != '-']
    dict_address_coord = dict(df_address_coord[['Адрес из детальной раскладки', 'Координаты nominatim']].to_numpy())
    df['Координаты nominatim'] = df['Адрес детальной раскладки'].apply(lambda x: dict_address_coord[x] if x in dict_address_coord else None)
    return df

from geopy.distance import geodesic

def calculate_distance(location1, location2):
    """Вычисляет расстояние между 2-мя точками по прямой."""
    distance = geodesic(location1, location2)
    return round(distance.km, 3)
    # df['Разница координат'] = df.apply(lambda row: calculate_distance(row['Координаты объекта'], row['Координаты nominatim']) if row['Координаты nominatim'] != None else None, axis=1)

## 5. Округляем координаты (для единообразия)

In [267]:
# df['Координаты объекта'] = df['Координаты объекта'].apply(lambda coord: (round(coord[0], 5), round(coord[1], 5)))

## 6. Создание уникальных объектов

In [268]:
# Определяем уникальность, это сцепка координаты + площадь

def unique_number(df):
    j = 0
    dict_unique_number = dict()
    for index, row in df.iterrows():
        string = str(row['Площадь объекта(м²)']) + str('_') + str(row['Координаты объекта'])
        if string not in dict_unique_number:
            j += 1
            dict_unique_number[string] = j
            df.at[index, 'Уникальный номер'] = j
        else:
            df.at[index, 'Уникальный номер'] = dict_unique_number[string]
    return df

df['Уникальный номер'] = pd.Series()
df = unique_number(df)

## 7. Сегментация объектов (выбросы по Тексту объявления)

In [269]:
# Константы
sheet_name_1 = 'торги, доли, обременения'
sheet_name_2 = 'ОНСы, под снос'
sheet_name_3 = 'С остатками товара'
sheet_name_4 = 'Продажа бизнеса'
sheet_name_5 = 'Здания'

In [270]:
# Фильтрация по столбцу 'Текст объявления'

# Пусть к Выбросам по словарю
path_dict = PATH_DICT_DATA + 'словарь_1.xlsx'

# Поверка на наличие пустых строк и их заполнение
def check_for_empty_strings(data_dict):
    if data_dict['Выбросы'].isna().any():
        data_dict['Выбросы'] = data_dict['Выбросы'].fillna('-')
    
    # Поверка на наличие пустых строк и их заполнение
    if data_dict['Исключения'].isna().any():
        data_dict['Исключения'] = data_dict['Исключения'].fillna('-')

def devide(data_dict, data_df):
    """Функция сегментации датафрейма по словарям."""
    check_for_empty_strings(data_dict)

    devide_list = data_dict['Выбросы'].to_list()

    # создание регулярного выражения
    pattern_devide = '|'.join(devide_list)

    exceptions_list = data_dict['Исключения'].to_list()
    pattern_exceptions = '|'.join(exceptions_list)

    # regex - параметр регулярного выражения
    condition_1 = data_df['Текст объявления'].str.contains(pattern_devide, regex=True)
    condition_2 = ~data_df['Текст объявления'].str.contains(pattern_exceptions, regex=True)

    data_df = data_df[condition_1 & condition_2]
    
    return data_df

In [271]:
# def # Сохранение выбросов

# Чтение и запись словаря в переменную
data_section_1 = pd.read_excel(path_dict, sheet_name=sheet_name_1)
data_section_2 = pd.read_excel(path_dict, sheet_name=sheet_name_2)
data_section_3 = pd.read_excel(path_dict, sheet_name=sheet_name_3)
data_section_4 = pd.read_excel(path_dict, sheet_name=sheet_name_4)
data_section_5 = pd.read_excel(path_dict, sheet_name=sheet_name_5)

# Сохранение сегментированного датафрейма в переменную
df_section_1 = devide(data_section_1, df)
df = df[~df.index.isin(df_section_1.index)]
# df_section_1.to_excel(PATH_CLEAN_DATA + 'ВЫБРОСЫ - ' + sheet_name_1 + '.xlsx', index=False)

# Статистика
control_count_temp = pd.pivot_table(df_section_1, 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': sheet_name_1})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)


df_section_2 = devide(data_section_2, df)
df = df[~df.index.isin(df_section_2.index)]
# df_section_2.to_excel(PATH_CLEAN_DATA + 'ВЫБРОСЫ - ' + sheet_name_2 + '.xlsx', index=False)

# Статистика
control_count_temp = pd.pivot_table(df_section_2, 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': sheet_name_2})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)


df_section_3 = devide(data_section_3, df)
df = df[~df.index.isin(df_section_3.index)]
# df_section_3.to_excel(PATH_CLEAN_DATA + 'ВЫБРОСЫ - ' + sheet_name_3 + '.xlsx', index=False)

# Статистика
control_count_temp = pd.pivot_table(df_section_3, 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': sheet_name_3})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)


df_section_4 = devide(data_section_4, df)
df = df[~df.index.isin(df_section_4.index)]
# df_section_4.to_excel(PATH_CLEAN_DATA + 'ВЫБРОСЫ - ' + sheet_name_4 + '.xlsx', index=False)

# Статистика
control_count_temp = pd.pivot_table(df_section_4, 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': sheet_name_4})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)


df_section_5 = devide(data_section_5, df)
df = df[~df.index.isin(df_section_5.index)]
# df_section_5.to_excel(PATH_CLEAN_DATA + 'ВЫБРОСЫ - ' + sheet_name_5 + '.xlsx', index=False)

# Статистика
control_count_temp = pd.pivot_table(df_section_5, 
                                      values='Дата объявления', 
                                      index='Имя файла',
                                      aggfunc='count').rename(columns = {'Дата объявления': sheet_name_5})

control_count_result = pd.merge(control_count_result, control_count_temp,
                                left_on = 'Имя файла',
                                right_on = 'Имя файла',
                                how = 'left').fillna(0)


df = df.reset_index(drop=True)

## 8. Сегментация объектов (выбросы по Заголовкам)

In [272]:
list_sentences = df['Заголовок'].unique()
list_key_words = list()


for sentence in list_sentences:
    temp = list()
    for term in term_extractor(sentence):
        if len(term.normalized) > 2: # исключаем то что состоит из одной или двух букв     
            temp.append(term.normalized)
    list_key_words.append(temp)

In [273]:
# приведение словосочетаний к начальной форме (для исключения вариативности)
list_basic_key_words = []
for keywords in list_key_words:
    temp = []
    for keyword in keywords:
        if len(keyword.split(' ')) > 1:
            for word in keyword.split(' '):
                basic_form = morph.parse(word)[0].normal_form
                if len(basic_form) > 2 and not basic_form.isdigit():
                    temp.append(basic_form)
        else:
            basic_form = morph.parse(keyword)[0].normal_form
            if len(basic_form) > 2 and not basic_form.isdigit():
                temp.append(basic_form)

    list_basic_key_words.append(temp)

In [274]:
temp_df = pd.DataFrame({
    "Заголовок": list_sentences,
    "Ключевые слова": list_key_words,
    "Нач.форма ключевых слов": list_basic_key_words,
})

In [275]:
# Константы 
path_keywords = PATH_DICT_DATA + 'словарь_2_ключевые_слова_по_заголовкам.xlsx'

sheet_name_building = 'Здания'
sheet_name_rooms = 'Помещения'
sheet_name_complexes = 'Комплексы'

data_building = pd.read_excel(path_keywords, sheet_name=sheet_name_building)
data_rooms = pd.read_excel(path_keywords, sheet_name=sheet_name_rooms)
data_complexes = pd.read_excel(path_keywords, sheet_name=sheet_name_complexes)

temp_df['Категория'] = pd.Series()

In [276]:
def define_category(df_list, data_dict_1, data_dict_2, data_dict_3):
    """Определяет вид недвижимости по ключевым словам в заголовке.

    Параметры:
    df_list (list): список слов из заголовка.
    data_dict_1 (DataFrame): Датафрейм ключевых слов для категории "Здания".
    data_dict_2 (DataFrame): Датафрейм ключевых слов для категории "Помещения".
    data_dict_3 (DataFrame): Датафрейм ключевых слов для категории "Комплексы".

    Возвращает:
    str: выбранная категория по словарям.
    Возможные значения:
        - 'Комплексы'
        - 'Здания'
        - 'Помещения'
        - '-' (если категория не найдена)
    """
    list_keywords_1 = data_dict_1['Ключевые слова'].to_list()
    list_keywords_2 = data_dict_2['Ключевые слова'].to_list()
    list_keywords_3 = data_dict_3['Ключевые слова'].to_list()

    output = []

    for word in df_list:
        for keyword in list_keywords_3:
            if word in keyword:
                return 'Комплексы'
        for keyword in list_keywords_1:
            if word in keyword:
                output.append('Здания')
        for keyword in list_keywords_2:
            if word in keyword:
                output.append('Помещения')

    if output:
        return sorted(output)[0]
    else:
        return '-'

In [277]:
temp_df['Категория'] = temp_df['Нач.форма ключевых слов'].apply(lambda x: define_category(x, data_building, data_rooms, data_complexes))

In [278]:
# temp_df.to_excel(PATH_CLEAN_DATA + 'Ключевые слова по заголовкам.xlsx', index=False)

## 9. Сбор итогового файла

In [279]:
category_dict = dict(temp_df[['Заголовок', 'Категория']].to_numpy())

In [280]:
df['Вид недвижимости'] = df['Заголовок'].progress_apply(lambda x: category_dict[x])

100%|█████████████████████████████████████████████████████████████████████████| 1401/1401 [00:00<00:00, 1427999.98it/s]


## Подбиваем итоги по выбросам

In [282]:
# Статистика
control_count_result['сумма по выбросам'] = control_count_result[list(control_count_result.columns[1:])].sum(axis=1)
control_count_result['итого'] = control_count_result['Исходное количество записей']-control_count_result['сумма по выбросам']
temp = pd.DataFrame(control_count_result[control_count_result.columns].sum(), columns = ['итого'])
temp = temp.T
control_count_result = pd.concat([control_count_result, temp])
control_count_result = control_count_result.reset_index().rename(columns = {'index': 'Имя файла'})
control_count_result['Дата выгрузки'] = control_count_result['Имя файла'].apply(lambda x: x.split('_')[1] if len(x.split('_')) > 1 else '')
control_count_result = control_count_result[['Имя файла', 'Дата выгрузки']+ list(control_count_result.columns[1:-1])]
control_count_result['Дата выгрузки'] = pd.to_datetime(control_count_result['Дата выгрузки'], dayfirst=True).dt.date

In [283]:
control_count_result.to_excel(PATH_CLEAN_DATA + 'Статистика по выбросам.xlsx', index=False)

In [288]:
df_23 = df[df['Имя файла'].str.contains('2023')]
df_24 = df[df['Имя файла'].str.contains('2024')]

df_23.to_excel(PATH_CLEAN_DATA + 'Итог 2023.xlsx', index=False)
df_24.to_excel(PATH_CLEAN_DATA + 'Итог 2024.xlsx', index=False)

In [289]:
df_clean_23 = df_23.drop_duplicates('Уникальный номер')
df_clean_24 = df_24.drop_duplicates('Уникальный номер')

df_clean_23.to_excel(PATH_CLEAN_DATA + 'Итог(без дублей) 2023.xlsx', index=False)
df_clean_24.to_excel(PATH_CLEAN_DATA + 'Итог(без дублей) 2024.xlsx', index=False)

In [ ]:
df_unique_23 = df_23.groupby('Имя файла').apply(lambda x: x.drop_duplicates(subset='Уникальный номер'), include_groups=False).reset_index(drop=True)
df_unique_24 = df_24.groupby('Имя файла').apply(lambda x: x.drop_duplicates(subset='Уникальный номер'), include_groups=False).reset_index(drop=True)

df_unique_23.to_excel(PATH_CLEAN_DATA + 'Итог(без дублей по месяцам) 2023.xlsx', index=False)
df_unique_24.to_excel(PATH_CLEAN_DATA + 'Итог(без дублей по месяцам) 2024.xlsx', index=False)